In [ ]:
%matplotlib inline


Introduction to PyTorch
***********************

Introduction to Torch's tensor library
======================================

All of deep learning is computations on tensors, which are
generalizations of a matrix that can be indexed in more than 2
dimensions. We will see exactly what this means in-depth later. First,
lets look what we can do with tensors.



In [1]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

Creating Tensors
~~~~~~~~~~~~~~~~

Tensors can be created from Python lists with the torch.Tensor()
function.




In [2]:
# torch.tensor(data) creates a torch.Tensor object with the given data.
V_data = [1., 2., 3.]
V = torch.tensor(V_data)
print(V)

# Creates a matrix
M_data = [[1., 2., 3.], [4., 5., 6]]
M = torch.tensor(M_data)
print(M)

# Create a 3D tensor of size 2x2x2.
T_data = [[[1., 2.], [3., 4.]],
          [[5., 6.], [7., 8.]]]
T = torch.tensor(T_data)
print(T)

tensor([1., 2., 3.])
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]])


What is a 3D tensor anyway? Think about it like this. If you have a
vector, indexing into the vector gives you a scalar. If you have a
matrix, indexing into the matrix gives you a vector. If you have a 3D
tensor, then indexing into the tensor gives you a matrix!

A note on terminology:
when I say "tensor" in this tutorial, it refers
to any torch.Tensor object. Matrices and vectors are special cases of
torch.Tensors, where their dimension is 1 and 2 respectively. When I am
talking about 3D tensors, I will explicitly use the term "3D tensor".




In [3]:
# Index into V and get a scalar (0 dimensional tensor)
print(V[0])
# Get a Python number from it
print(V[0].item())

# Index into M and get a vector
print(M[0])

# Index into T and get a matrix
print(T[0])

tensor(1.)
1.0
tensor([1., 2., 3.])
tensor([[1., 2.],
        [3., 4.]])


You can also create tensors of other datatypes. The default, as you can
see, is Float. To create a tensor of integer types, try
torch.LongTensor(). Check the documentation for more data types, but
Float and Long will be the most common.




You can create a tensor with random data and the supplied dimensionality
with torch.randn()




In [4]:
x = torch.randn((3, 4, 5))
print(x)

tensor([[[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002],
         [-0.6092, -0.9798, -1.6091, -0.7121,  0.3037],
         [-0.7773, -0.2515, -0.2223,  1.6871,  0.2284],
         [ 0.4676, -0.6970, -1.1608,  0.6995,  0.1991]],

        [[ 0.8657,  0.2444, -0.6629,  0.8073,  1.1017],
         [-0.1759, -2.2456, -1.4465,  0.0612, -0.6177],
         [-0.7981, -0.1316,  1.8793, -0.0721,  0.1578],
         [-0.7735,  0.1991,  0.0457,  0.1530, -0.4757]],

        [[-0.1110,  0.2927, -0.1578, -0.0288,  0.4533],
         [ 1.1422,  0.2486, -1.7754, -0.0255, -1.0233],
         [-0.5962, -1.0055,  0.4285,  1.4761, -1.7869],
         [ 1.6103, -0.7040, -0.1853, -0.9962, -0.8313]]])


Operations with Tensors
~~~~~~~~~~~~~~~~~~~~~~~

You can operate on tensors in the ways you would expect.



In [6]:
x = torch.tensor([1., 2., 3.])
y = torch.tensor([4., 5., 6.])
z = x + y
z = torch.add(x, y)
print(z)

tensor([5., 7., 9.])


See `the documentation <http://pytorch.org/docs/torch.html>`__ for a
complete list of the massive number of operations available to you. They
expand beyond just mathematical operations.

One helpful operation that we will make use of later is concatenation.




In [9]:
# By default, it concatenates along the first axis (concatenates rows)
x_1 = torch.randn(2, 5)
y_1 = torch.randn(3, 5)
z_1 = torch.cat([x_1, y_1])
print(z_1)
print(z_1.shape)

# Concatenate columns:
x_2 = torch.randn(2, 3)
y_2 = torch.randn(2, 5)
# second arg specifies which axis to concat along
z_2 = torch.cat([x_2, y_2], 1)
print(z_2)

# If your tensors are not compatible, torch will complain.  Uncomment to see the error
torch.cat([x_1, x_2])

tensor([[-0.3339, -1.4724,  0.7296, -0.1312, -0.6368],
        [ 1.0429,  0.4903,  1.0318, -0.5989,  1.6015],
        [-1.0735, -1.2173,  0.6472, -0.0412, -0.1775],
        [-0.5000,  0.8673, -0.2732, -0.4608, -0.0991],
        [ 0.4728,  1.0049, -0.2871, -1.1619,  0.0276]])
torch.Size([5, 5])
tensor([[ 0.5652, -0.0115,  0.6706,  1.6169, -0.9026,  0.1737,  0.0772, -0.9339],
        [-0.4929,  1.5050, -2.3264,  0.0914,  1.3940, -0.6877, -0.5058,  0.1924]])


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 5 and 3 in dimension 1 at /home/adam/code/pytorch/pytorch/aten/src/TH/generic/THTensorMath.cpp:3581

Reshaping Tensors
~~~~~~~~~~~~~~~~~

Use the .view() method to reshape a tensor. This method receives heavy
use, because many neural network components expect their inputs to have
a certain shape. Often you will need to reshape before passing your data
to the component.




In [10]:
x = torch.randn(2, 3, 4)
print(x)
print(x.view(2, 12))  # Reshape to 2 rows, 12 columns
# Same as above.  If one of the dimensions is -1, its size can be inferred
print(x.view(2, -1))

tensor([[[ 1.4023,  0.4841, -0.7030, -0.8268],
         [ 0.1012,  0.1952, -1.1796, -1.4423],
         [ 0.2936, -0.4139, -0.0960, -1.3281]],

        [[ 0.2324,  0.8615,  0.6218, -1.7812],
         [-0.0339, -0.5396, -1.8074,  0.3425],
         [ 0.8532,  0.0551, -1.7425,  0.8750]]])
tensor([[ 1.4023,  0.4841, -0.7030, -0.8268,  0.1012,  0.1952, -1.1796, -1.4423,
          0.2936, -0.4139, -0.0960, -1.3281],
        [ 0.2324,  0.8615,  0.6218, -1.7812, -0.0339, -0.5396, -1.8074,  0.3425,
          0.8532,  0.0551, -1.7425,  0.8750]])
tensor([[ 1.4023,  0.4841, -0.7030, -0.8268,  0.1012,  0.1952, -1.1796, -1.4423,
          0.2936, -0.4139, -0.0960, -1.3281],
        [ 0.2324,  0.8615,  0.6218, -1.7812, -0.0339, -0.5396, -1.8074,  0.3425,
          0.8532,  0.0551, -1.7425,  0.8750]])


Computation Graphs and Automatic Differentiation
================================================

The concept of a computation graph is essential to efficient deep
learning programming, because it allows you to not have to write the
back propagation gradients yourself. A computation graph is simply a
specification of how your data is combined to give you the output. Since
the graph totally specifies what parameters were involved with which
operations, it contains enough information to compute derivatives. This
probably sounds vague, so let's see what is going on using the
fundamental flag ``requires_grad``.

First, think from a programmers perspective. What is stored in the
torch.Tensor objects we were creating above? Obviously the data and the
shape, and maybe a few other things. But when we added two tensors
together, we got an output tensor. All this output tensor knows is its
data and shape. It has no idea that it was the sum of two other tensors
(it could have been read in from a file, it could be the result of some
other operation, etc.)

If ``requires_grad=True``, the Tensor object keeps track of how it was
created. Lets see it in action.




In [12]:
# Tensor factory methods have a ``requires_grad`` flag
x = torch.tensor([1., 2., 3], requires_grad=True)

# With requires_grad=True, you can still do all the operations you previously
# could
y = torch.tensor([4., 5., 6], requires_grad=True)
z = x + y
print(z)

# BUT z knows something extra.
print(z.grad_fn)

x_no_grad = torch.tensor([1., 2., 3.], requires_grad=False)
y_no_grad = torch.tensor([4., 5., 6.], requires_grad=False)
z_no_grad = x_no_grad + y_no_grad
print(z_no_grad)
print(z_no_grad.grad_fn)


tensor([5., 7., 9.], grad_fn=<ThAddBackward>)
tensor([5., 7., 9.])
None


So Tensors know what created them. z knows that it wasn't read in from
a file, it wasn't the result of a multiplication or exponential or
whatever. And if you keep following z.grad_fn, you will find yourself at
x and y.

But how does that help us compute a gradient?




In [13]:
# Lets sum up all the entries in z
s = z.sum()
print(s)
print(s.grad_fn)

tensor(21., grad_fn=<SumBackward0>)


So now, what is the derivative of this sum with respect to the first
component of x? In math, we want

\begin{align}\frac{\partial s}{\partial x_0}\end{align}

Well, s knows that it was created as a sum of the tensor z. z knows
that it was the sum x + y. So

\begin{align} s = \overbrace{x_0 + y_0}^\text{$z_0$} + \overbrace{x_1 + y_1}^\text{$z_1$} + \overbrace{x_2 + y_2}^\text{$z_2$} \end{align}

And so s contains enough information to determine that the derivative
we want is 1!

Of course this glosses over the challenge of how to actually compute
that derivative. The point here is that s is carrying along enough
information that it is possible to compute it. In reality, the
developers of Pytorch program the sum() and + operations to know how to
compute their gradients, and run the back propagation algorithm. An
in-depth discussion of that algorithm is beyond the scope of this
tutorial.




Lets have Pytorch compute the gradient, and see that we were right:
(note if you run this block multiple times, the gradient will increment.
That is because Pytorch *accumulates* the gradient into the .grad
property, since for many models this is very convenient.)




In [ ]:
# calling .backward() on any variable will run backprop, starting from it.
s.backward()
print(x.grad)

Understanding what is going on in the block below is crucial for being a
successful programmer in deep learning.




In [15]:
x = torch.randn(2, 2)
y = torch.randn(2, 2)
# By default, user created Tensors have ``requires_grad=False``
print(x.requires_grad, y.requires_grad)
z = x + y
# So you can't backprop through z
print(z.grad_fn)

# ``.requires_grad_( ... )`` changes an existing Tensor's ``requires_grad``
# flag in-place. The input flag defaults to ``True`` if not given.
x = x.requires_grad_()
y = y.requires_grad_()
# z contains enough information to compute gradients, as we saw above
z = x + y
print(z.grad_fn)
# If any input to an operation has ``requires_grad=True``, so will the output
print(z.requires_grad)

# Now z has the computation history that relates itself to x and y
# Can we just take its values, and **detach** it from its history?
new_z = z.detach()

# ... does new_z have information to backprop to x and y?
# NO!
print(new_z.grad_fn)
# And how could it? ``z.detach()`` returns a tensor that shares the same storage
# as ``z``, but with the computation history forgotten. It doesn't know anything
# about how it was computed.
# In essence, we have broken the Tensor away from its past history

False False
None
True
None


You can also stop autograd from tracking history on Tensors
with ``.requires_grad``=True by wrapping the code block in
``with torch.no_grad():``



In [16]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False
